In [6]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from numpy import genfromtxt
import numpy as np


In [7]:
class MyMusicDataset(Dataset):
    def __init__(self, data_file_path, insert_random_data = True, n_classes = 26): # [0, 25]
        np_data = genfromtxt(data_file_path, dtype = np.int32)
        #print(np_data.shape)
        np_data_y = np.ones_like(np_data[:, 0:1]) # full score, input scores are discarded
        np_data_x = np_data[:, 1:] # sequence of music notes

        data_x = torch.Tensor(np_data_x) # integer class numbers in [0, 25]
        data_y = torch.Tensor(np_data_y) # scores are in {0, 1}
        
        
        if (insert_random_data):
            random_x = torch.randint(low = 0, high = n_classes, size = data_x.shape, dtype = torch.int)# [0, 25]
            random_y = torch.zeros_like(data_y) # random music is bad music
            data_x = torch.cat((data_x, random_x))
            data_y = torch.cat((data_y, random_y))
        data_x = torch.unsqueeze(data_x,1)
        
        # one hot representation
        #data_x = F.one_hot(data_x.to(torch.int64), num_classes = n_classes)
        print(data_x.shape) #(N,C,seqlen,num_classes)
        self.data_x = data_x.double()
        self.data_y = data_y.double()
        #print(data_y)
    
    def __len__(self):
        return len(self.data_y)
    def __getitem__(self, idx):
        return self.data_x[idx], self.data_y[idx] # seqs, labels



In [8]:
class MusicNet(nn.Module):
    def __init__(self, seq_len):
        super(MusicNet, self).__init__()         # N, C(annnels), L(ength of the sequence)
#         self.extractor = nn.Sequential(
#             nn.Flatten(start_dim = 2, end_dim = 3),#(N,C=1,seqlen*num_classes) 
#             nn.Conv1d(in_channels = 1, out_channels = 2, kernel_size = 52), 
#             nn.AvgPool1d(kernel_size = 4),
#             nn.Tanh(),
#             nn.Conv1d(in_channels = 2, out_channels = 2, kernel_size = 26), 
#             nn.Tanh(),
#             nn.AvgPool1d(kernel_size = 4),
#             nn.Conv1d(in_channels = 2, out_channels = 5, kernel_size = 26), 
#             nn.AvgPool1d(kernel_size = 4),
#             nn.Tanh(),
#             nn.Flatten(),
#             nn.Linear(25, 20),
#             nn.Tanh(),
#             nn.Linear(20, 1),
#             nn.Sigmoid()
#         )
        self.extractor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 200),
            nn.Sigmoid(),
            nn.Linear(200, 10),
            nn.Sigmoid(),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        #print(x.shape)
        return self.extractor(x)

In [64]:
def train(model, train_loader, num_epochs, loss_fn, optimizer):
    for epoch in range(0, num_epochs):
        print("epoch", epoch)
        for batch, (X, y) in enumerate(train_loader):
            for rep in range(1):
                pred = model(X.float())
                loss = loss_fn(pred.float(), y.float())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                if (batch % 10 == 0):
                    print("y: ", y)
                    print("pred: ", pred)
                    print("loss: ", loss)

def test(model, test_loader, loss_fn):
    losses = []
    for batch, (X, y) in enumerate(test_loader):
        pred = model(X.float())
        loss = loss_fn(pred.float(), y.float())
        losses.append(float(loss))
        print("%.6f" % float(loss))
    print("total avg loss", sum(losses) / len(losses))

In [10]:
# test the dataloader
train_data = MyMusicDataset('./data/chopin_nocturnes_complete.txt', insert_random_data = True, n_classes = 28)
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
train_features, train_labels = next(iter(train_dataloader))
print(train_features)
print(train_labels)

torch.Size([1272, 1, 32])
tensor([[[25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
          25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 24., 24., 24., 24.,
          25., 25., 17., 17.]],

        [[21.,  5., 23.,  7., 22., 16.,  4., 19., 21., 23.,  2., 23., 20.,  0.,
          15., 12., 25., 26., 23., 15.,  9., 21., 26., 25., 27., 20.,  8., 25.,
          10., 20.,  7., 12.]],

        [[ 6.,  9., 26., 22., 17.,  1., 14., 13., 23.,  3., 27.,  4., 22., 12.,
          26.,  2.,  7., 27., 13.,  8., 24., 17., 19., 22., 19.,  1.,  0., 15.,
          19., 26., 16.,  8.]],

        [[24., 26., 24., 24., 23., 24., 21., 21., 24., 24., 23., 23., 23., 21.,
          19., 18., 16., 19., 18., 21., 18., 19., 16., 16., 19., 19., 19., 18.,
          23., 21., 19., 18.]],

        [[20.,  0.,  1., 10., 14., 11., 24., 22., 20.,  4., 12., 24., 20., 25.,
          24.,  8., 25., 17., 11., 21.,  1., 15., 11., 19.,  7.,  8., 22., 26.,
           8., 11.,  6., 17.]],

        [

In [51]:
# create & test the network

net = MusicNet(32)
train_features, train_labels = next(iter(train_dataloader))
print(net(train_features.float()))
print(train_labels)

tensor([[0.4073],
        [0.4147],
        [0.4213],
        [0.3987],
        [0.4113],
        [0.4216],
        [0.4008],
        [0.4109],
        [0.4032],
        [0.4073],
        [0.4092],
        [0.4102],
        [0.4167],
        [0.4087],
        [0.4102],
        [0.4173]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]], dtype=torch.float64)


In [62]:
# prepare dataset
train_data = MyMusicDataset('./data/chopin_nocturnes_train.txt', insert_random_data = True, n_classes = 28)
test_data = MyMusicDataset('./data/chopin_nocturnes_test.txt', insert_random_data = True, n_classes = 28)

train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

torch.Size([1072, 1, 32])
torch.Size([200, 1, 32])


In [68]:
# start the training process
loss_fn = nn.MSELoss()
#loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.05)
train(net, train_dataloader, 500, loss_fn, optimizer)

epoch 0
y:  tensor([[0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.0193],
        [0.0185],
        [0.9985],
        [0.9895],
        [0.9858],
        [0.9969],
        [0.0022],
        [0.9997],
        [0.9997],
        [0.0026],
        [0.9992],
        [0.0074],
        [0.0100],
        [0.0107],
        [0.0014],
        [0.9815]], grad_fn=<SigmoidBackward0>)
loss:  tensor(0.0001, grad_fn=<MseLossBackward0>)
y:  tensor([[1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.9741],
        [0.0200],
        [0.0219],
        [0.9863],
        [0.9997],
        [0.0136],
   

y:  tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.9949],
        [0.9876],
        [0.9814],
        [0.0025],
        [0.0127],
        [0.0019],
        [0.9998],
        [0.9869],
        [0.9994],
        [0.9906],
        [0.9942],
        [0.9936],
        [0.9998],
        [0.0076],
        [0.0126],
        [0.0156]], grad_fn=<SigmoidBackward0>)
loss:  tensor(9.3274e-05, grad_fn=<MseLossBackward0>)
y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[5.3133e-04],
        [9.8519e-01],
        [1.3401e-03],
        [1.2721e-02],
        [9.9127e-01],
     

y:  tensor([[0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)
pred:  tensor([[1.5951e-02],
        [8.4611e-03],
        [1.3765e-03],
        [4.0585e-04],
        [9.8630e-01],
        [9.5415e-03],
        [1.8542e-02],
        [9.9979e-01],
        [9.9979e-01],
        [9.9691e-01],
        [9.9981e-01],
        [9.8477e-01],
        [9.9979e-01],
        [1.4170e-02],
        [1.3733e-03],
        [1.6489e-02]], grad_fn=<SigmoidBackward0>)
loss:  tensor(0.0001, grad_fn=<MseLossBackward0>)
y:  tensor([[0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[1.4036e-02],
        [9.5386e-03],
        [9.

y:  tensor([[1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.9759],
        [0.0025],
        [0.0022],
        [0.9893],
        [0.9984],
        [0.9829],
        [0.9906],
        [0.0013],
        [0.0130],
        [0.9841],
        [0.0010],
        [0.0138],
        [0.0099],
        [0.0180],
        [0.9994],
        [0.9837]], grad_fn=<SigmoidBackward0>)
loss:  tensor(0.0001, grad_fn=<MseLossBackward0>)
y:  tensor([[1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]], dtype=torch.float64)
pred:  tensor([[9.8118e-01],
        [1.6394e-02],
        [9.7690e-01],
        [1.1208e-02],
        [6.2384e-04],
        [

y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.0056],
        [0.9699],
        [0.0185],
        [0.0017],
        [0.0123],
        [0.9993],
        [0.9880],
        [0.0029],
        [0.9951],
        [0.0079],
        [0.0069],
        [0.9981],
        [0.9850],
        [0.0025],
        [0.9909],
        [0.9847]], grad_fn=<SigmoidBackward0>)
loss:  tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch 23
y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.]], dtype=torch.float64)
pred:  tensor([[2.1950e-03],
        [9.9129e-01],
        [9.3333e-03],
        [5.2176e-04],
        [9.9813e-01],


...

y:  tensor([[1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[9.9344e-01],
        [1.0308e-03],
        [3.6767e-03],
        [1.5618e-04],
        [8.1205e-03],
        [4.7413e-03],
        [3.3380e-03],
        [6.0066e-03],
        [8.8883e-03],
        [4.4466e-03],
        [9.9345e-01],
        [9.9980e-01],
        [9.9972e-01],
        [9.9980e-01],
        [9.8644e-01],
        [1.1176e-02]], grad_fn=<SigmoidBackward0>)
loss:  tensor(4.0251e-05, grad_fn=<MseLossBackward0>)
y:  tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.]], dtype=torch.float64)
pred:  tensor([[9.3806e-03],
        [6.1672e-03],
       

epoch 480
y:  tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=torch.float64)
pred:  tensor([[9.9968e-01],
        [6.7523e-03],
        [9.9982e-01],
        [7.9070e-03],
        [9.9304e-01],
        [8.3415e-03],
        [5.1085e-04],
        [9.9196e-01],
        [5.7999e-04],
        [5.1627e-03],
        [9.8810e-01],
        [9.9870e-01],
        [9.8043e-01],
        [9.9380e-01],
        [9.8221e-01],
        [9.9978e-01]], grad_fn=<SigmoidBackward0>)
loss:  tensor(7.4951e-05, grad_fn=<MseLossBackward0>)
y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.]], dtype=torch.float64)
pred:  tensor([[1.0934e-02],
        [9.9985e-01

epoch 486
y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)
pred:  tensor([[6.2471e-03],
        [9.9104e-01],
        [9.0739e-03],
        [1.0894e-02],
        [1.5094e-02],
        [9.9379e-01],
        [9.9981e-01],
        [1.3333e-02],
        [6.0256e-04],
        [9.9975e-01],
        [9.9982e-01],
        [9.9982e-01],
        [9.9982e-01],
        [9.4292e-03],
        [5.8954e-03],
        [4.3567e-03]], grad_fn=<SigmoidBackward0>)
loss:  tensor(5.6730e-05, grad_fn=<MseLossBackward0>)
y:  tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.0118],
        [0.0113],
     

epoch 492
y:  tensor([[0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]], dtype=torch.float64)
pred:  tensor([[6.6586e-03],
        [9.9984e-01],
        [9.8956e-01],
        [9.9610e-01],
        [8.4155e-04],
        [8.1393e-03],
        [7.0730e-03],
        [1.9911e-03],
        [9.3345e-03],
        [9.9898e-01],
        [9.9984e-01],
        [5.7436e-04],
        [7.4424e-03],
        [9.9058e-01],
        [9.9414e-01],
        [9.9931e-01]], grad_fn=<SigmoidBackward0>)
loss:  tensor(3.4813e-05, grad_fn=<MseLossBackward0>)
y:  tensor([[0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.0059],
        [0.0021],
     

y:  tensor([[0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[1.1840e-02],
        [9.9118e-01],
        [9.9412e-01],
        [9.9004e-01],
        [9.9951e-01],
        [9.9947e-01],
        [6.7492e-04],
        [9.9468e-01],
        [9.8680e-01],
        [9.9970e-01],
        [3.7523e-03],
        [1.9858e-03],
        [9.9224e-01],
        [1.1644e-02],
        [9.9768e-01],
        [7.9313e-03]], grad_fn=<SigmoidBackward0>)
loss:  tensor(5.2348e-05, grad_fn=<MseLossBackward0>)
epoch 498
y:  tensor([[1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.9904],
        [0.0057],
     

In [69]:
test(net, test_dataloader, loss_fn)

0.000105
0.000003
0.000075
0.000000
0.930980
0.000051
0.000353
0.000000
0.000013
0.420787
0.885226
0.000023
0.000093
0.000001
0.983223
0.003513
0.012333
0.000000
0.000392
0.000001
0.000000
0.000003
0.000001
0.000752
0.006783
0.000000
0.000001
0.000001
0.000000
0.000000
0.000571
0.000307
0.911329
0.000765
0.000078
0.002578
0.000000
0.993347
0.000002
0.159809
0.000479
0.000002
0.000001
0.000001
0.000001
0.000002
0.001733
0.000000
0.000000
0.000000
0.930481
0.990525
0.000000
0.000499
0.000000
0.005979
0.000003
0.359842
0.000000
0.000017
0.954054
0.000002
0.882681
0.000012
0.000000
0.972443
0.000000
0.989878
0.000001
0.000000
0.000019
0.000000
0.000026
0.000002
0.000000
0.000547
0.000005
0.000013
0.000648
0.006681
0.000000
0.000000
0.000000
0.993756
0.000001
0.000000
0.000043
0.052412
0.000024
0.001138
0.000000
0.000000
0.000001
0.000006
0.112965
0.000000
0.000006
0.000013
0.000009
0.000000
0.000139
0.000018
0.781620
0.000000
0.001325
0.002537
0.000003
0.000004
0.001184
0.000000
0.000016
0